In [4]:
from kmodes.kmodes import KModes
import numpy as np
import pandas as pd

file = '../Data/東森/20250407_標籤名單.xlsx'
df = pd.read_excel(file, sheet_name='rawdata')

label_names = df['標籤名稱'].unique()
df_simple = df[['客代', '標籤名稱', '標籤值']]

df_pivot = df_simple.pivot_table(index='客代',
                                 columns='標籤名稱',
                                 values='標籤值',
                                 aggfunc=lambda x: '、'.join(map(str, x.unique()))) 

df_pivot = df_pivot.reset_index()
selected_columns = ['會員等級', '是否有高健康意識', '保健瀏覽', '職業']

df_pivot = df_pivot.fillna('未知')
df_pivot_cluster = df_pivot[selected_columns].copy()

costs = []
models = []
K = range(2, 11)

for k in K:
    kmodes = KModes(n_clusters=k, init='Huang', n_init=5, verbose=0)
    kmodes.fit(df_pivot_cluster)
    costs.append(kmodes.cost_)
    models.append(kmodes)


p1 = np.array([K[0], costs[0]])
p2 = np.array([K[-1], costs[-1]])

def distance_to_line(p, p1, p2):
    return np.abs(np.cross(p2-p1, p1-p)) / np.linalg.norm(p2-p1)

distances = [distance_to_line(np.array([k, cost]), p1, p2) for k, cost in zip(K, costs)]

optimal_idx = np.argmax(distances)
optimal_k = K[optimal_idx]

kmodes_final = models[optimal_idx]
clusters = kmodes_final.predict(df_pivot_cluster)
df_pivot_cluster['群編號'] = clusters
df_pivot['群編號'] = clusters
df_pivot_cluster.to_csv('Cluster.csv', index=False, encoding='utf-8-sig')
df_pivot.to_csv('Customer_Data_cluster.csv', index=False, encoding='utf-8-sig')
